<a href="https://colab.research.google.com/github/Rafael-Pereira-Silva/Checkpoint_Cybersecurity/blob/main/Checkpoint_Cybersecurity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q pyngrok
!pip install -q streamlit
!pip install -q streamlit_ace
!pip install fuzzywuzzy[speedup]
!streamlit run app.py & > / dev / null &

In [1]:
%%writefile app.py
import re
import streamlit as st
import requests as req
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from pyngrok import ngrok

def regexlist(lista):

    a='(?i)'
    for i in lista:
        a += f"({i})|"
    a = a[:-1]
    return a


def api_nome():
    lista_nome = []
    lista_interpol = req.get("https://cspinheiro.github.io/interpol.json")
    interpol = lista_interpol.json()['interpol']
    for i in interpol:
        lista_nome.append(i['interpol'])
    return lista_nome

def api_passaport():
    lista_passaporte = []
    lista_bel = req.get("https://rafael-pereira-silva.github.io/bel.json")
    bel = lista_bel.json()['bel']
    for i in bel:
        lista_passaporte.append(str(i['bel']))
    return lista_passaporte


def getmatch_nome(input,lista):
    search = process.extractOne(input,lista,scorer=fuzz.token_sort_ratio)
    if search[1] > 75:
      return 1
    return 0
        

def valid_nome(input_nome,lista_nome):
  if len(input_nome) > 3 and len(re.split('\s',input_nome)) >= 2:
      nome = getmatch_nome(input_nome,lista_nome)
      if nome == 1:
        st.error('Nome suspeito, investigue o passageiro')
      if nome == 0:
        st.success('Nome verificado, o passageiro não é um suspeito')
  else:
      st.warning('Preencha o Nome novamente')

def getmatch_passa(input,lista):
    match = re.search(lista,input)
    if match:
        return 1
    else:
        return 0

def main():
    html_temp = """ <div style ="background-color:blue;padding:13px">
                      <h1 style = "color:white;text-align:center">Federale politie</h1>
                    </div>
                """

    st.markdown(html_temp, unsafe_allow_html = True)
    lista_passaporte = api_passaport()
    lista_nome = api_nome()
    lista_passaporte= regexlist(lista_passaporte)
    input_passaporte = st.text_input('Informe o número do Passaporte: ')
    input_nome = st.text_input('Informe o Nome Completo: ')
    input_passaporte = re.sub('\D', '', input_passaporte)

    if st.button("Verificar"):
      clear = 0
      clear = valida_verificado(input_passaporte,lista_passaporte)
      if clear == 1:
        valid_nome(input_nome,lista_nome)

def valida_verificado(input_passaporte,lista_passaporte):
      if len(input_passaporte) == 9:
        verificado = getmatch_passa(input_passaporte,lista_passaporte)
        if verificado == 1:
            st.error('Este passaporte é proibido, prenda o suspeito')
            return 0
        else:
          return 1
      else:
        st.warning('Preencha o passaporte novamente')

if __name__ == '__main__':
    main()

Writing app.py


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501

/bin/bash: streamlit: command not found
npx: installed 22 in 1.777s
your url is: https://happy-masks-sniff-34-83-75-87.loca.lt
